##### Copyright 2019 The TensorFlow Authors.

This notebook compares RNN (from TensorFlow website) with a Markov Chain Model and sees pretty similar performance on a Shakespeare text.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


### Read the data

First, look in the text:

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [5]:
''.join(sorted(list(set(text))))

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [8]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2022-03-16 11:36:12.385875: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [12]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(30):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i
z
e
n
:


B
e
f
o
r
e
 
w
e
 
p
r
o
c
e


In [18]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())
  print(len(seq))

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
101
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
101
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
101
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
101
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
101


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


2022-03-16 11:36:18.157691: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [25]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [26]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [30]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  16896     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  67650     
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [31]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 66), dtype=float32, numpy=
array([[-0.00902414,  0.00209395, -0.00055931, ...,  0.00049885,
        -0.00553386, -0.01809038],
       [ 0.00193806, -0.01038724, -0.00849163, ...,  0.00196755,
        -0.00749912, -0.00704062],
       [ 0.00987061, -0.00826642,  0.00504532, ..., -0.00343576,
         0.00128602, -0.00933781],
       ...,
       [ 0.01613344,  0.00296722, -0.00599359, ...,  0.00902608,
        -0.01703041,  0.00989005],
       [ 0.00990328,  0.01178147, -0.00317648, ...,  0.00167213,
        -0.00921695,  0.00016148],
       [-0.00266305,  0.00943312, -0.01288425, ...,  0.01017887,
        -0.00907951, -0.00545113]], dtype=float32)>

In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [33]:
sampled_indices

array([40, 27, 40, 19, 50, 18,  7, 64, 64, 49, 15, 11, 65, 56, 14, 60, 65,
       21, 62, 30, 55, 34, 20, 59, 45, 32, 64, 45, 61, 58, 30, 61, 31, 48,
       61, 57, 56, 43, 48,  6, 18, 14, 12, 57, 44, 48, 45, 50, 27, 38,  3,
       48, 60,  6, 41, 26, 43, 31, 44, 57, 34, 11, 25, 40, 30, 63, 16,  0,
       32, 65, 63, 14, 55, 40, 19, 37, 24, 58, 22, 52, 51, 58, 10, 45, 17,
        9, 36, 12,  9, 39, 28, 30, 60, 40, 27, 36, 13, 51, 60, 37])

Decode these to see the text predicted by this untrained model:

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'rt?\nSound drums and trumpets! farewell sour annoy!\nFor here, I hope, begins our lasting joy.\n\nARCHID'

Next Char Predictions:
 b"aNaFkE,yyjB:zqAuzHwQpUGtfSyfvsQvRivrqdi'EA;reifkNY!iu'bMdRerU:LaQxC[UNK]SzxApaFXKsImls3fD.W;.ZOQuaNW?luX"


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189739, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [37]:
tf.exp(example_batch_mean_loss).numpy()

66.00558

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [38]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [39]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [40]:
EPOCHS = 10

In [41]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 422s 2s/step - loss: 2.6953
Epoch 2/10
172/172 [==============================] - 417s 2s/step - loss: 1.9729
Epoch 3/10
172/172 [==============================] - 418s 2s/step - loss: 1.6974
Epoch 4/10
172/172 [==============================] - 419s 2s/step - loss: 1.5380
Epoch 5/10
172/172 [==============================] - 416s 2s/step - loss: 1.4394
Epoch 6/10
172/172 [==============================] - 418s 2s/step - loss: 1.3719
Epoch 7/10
172/172 [==============================] - 416s 2s/step - loss: 1.3194
Epoch 8/10
172/172 [==============================] - 420s 2s/step - loss: 1.2738
Epoch 9/10
172/172 [==============================] - 422s 2s/step - loss: 1.2315
Epoch 10/10
172/172 [==============================] - 423s 2s/step - loss: 1.1908


## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [42]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [43]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [44]:
start = time.time()
states = None
next_char = tf.constant(['I am very tired. '])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

I am very tired. Did not I die for comp
That Most auginualy sense shrewdly 't, but assore all one,
As I do not, let him will both it me, and we may contented
That six and Romeo to said he measure to him;
A grave-benest than forgive and many a great reepinish
Than they's to a son wrong'd out.

RICHARD:
Ah, sir, now my lords, bemoved me untern's wigo;
I have no more perished by the sea,
And to my fortune fair Katharina!
I have told you alone, I saw have them all;
With ressmants, overmore lit on yon, post your son.

CAMILLO:
You know him now, nor
Forbext thou canst.

LEONTES:
Breaken! poor sovereign, what slack in deadful cold?

USTird Gentleman:
That, good queen, help, hear me in your queen's:
As do now Deain were the rather vice,
And Jove of cabit, nor devils thou thinks
Shall have no more men else, and pluck my pound,
I will se fair lives: all so some funishies
That he comes to be this vowel, seek.

WARWICK:
Then, in my grace, my Lord of Flate: who,
But only combined thousand Tiest; me

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [45]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nStay, well, I would to God's sky, most help,\nBut what dot twenty heirer prosperity!\nThird Glumio, this night, who comes can time is\nWell Katch: and, uncle Glorduse of matroy to petsail.\nI hope these metil that it may be beg, and mad\nher not to a mealth: how keep nothings of merry!\n\nISABELLA:\nIf you know me as thou shatter'd?\nsteel, with that he would have govern'd in\nthe dam, people; I have poison'd as well,\nWhen he's cendening on the Duke of Henry's go all.\n\nABHORSON:\nBreak of my dogs, to her heart throne:\nLet him cut it upon this knavery: I'll go.\n\nKATHARINA:\nI am generate you.\n\nDUKE VINCENTIO:\nThis scater father.\n\nGRUMIO:\nI prothes so are thought to make him slew his victuous.\n\nLADY ANNE:\nWomen he was well aspectated; fet on you,\nThe dove-night; but yet, it claimed upon madam?\n\nSecond Murderer:\nWhy, then, I do beseech you; if you talk of you.\n\nCOMINIUS:\nO does convileding here!\n\nAUTOLYCUS:\nWe will deserve you take it else: s

## Markov Chain Method

Do a 1 letter prediction based on the previous n characters

- **** a b c d e f g h i j k . . .
- [aa] 0 0 0 0 0 0 0 0 0 0 0 0 0 0
- [ab] 1 0 0 0 0 0 0 0 0 0 0 0 0 0
- [ac]

Count the occurences of the next character in sets of (n+1) characters, and update the frequency array

When sampling, just sample the next character based on a lookup table of the first n characters.

In [46]:
len(text)
uniq = list(set(text))
print(uniq)

['A', 'D', 'o', 'c', '&', '$', 'p', 'u', 'Y', 'r', ';', 'K', 'F', 'U', 'Q', 'N', 'j', 'x', ' ', 'Z', 'B', 'y', 'E', 'l', 'q', 'e', ',', 'm', 'T', 'X', '.', 'f', 'v', 'h', 'W', 'z', '?', '-', 'k', 'P', 'd', 'V', 'i', 'O', '\n', 'g', '!', 'w', 'C', 'n', '3', ':', 's', "'", 'b', 'a', 'S', 'J', 'M', 'R', 't', 'G', 'I', 'H', 'L']


In [47]:
b = {x:0 for x in uniq}
b['x'] += 1
print(b)

{'A': 0, 'D': 0, 'o': 0, 'c': 0, '&': 0, '$': 0, 'p': 0, 'u': 0, 'Y': 0, 'r': 0, ';': 0, 'K': 0, 'F': 0, 'U': 0, 'Q': 0, 'N': 0, 'j': 0, 'x': 1, ' ': 0, 'Z': 0, 'B': 0, 'y': 0, 'E': 0, 'l': 0, 'q': 0, 'e': 0, ',': 0, 'm': 0, 'T': 0, 'X': 0, '.': 0, 'f': 0, 'v': 0, 'h': 0, 'W': 0, 'z': 0, '?': 0, '-': 0, 'k': 0, 'P': 0, 'd': 0, 'V': 0, 'i': 0, 'O': 0, '\n': 0, 'g': 0, '!': 0, 'w': 0, 'C': 0, 'n': 0, '3': 0, ':': 0, 's': 0, "'": 0, 'b': 0, 'a': 0, 'S': 0, 'J': 0, 'M': 0, 'R': 0, 't': 0, 'G': 0, 'I': 0, 'H': 0, 'L': 0}


In [48]:
import numpy as np

def sample(input_dict):
  value_counts = []
  char_array = []
  for key,value in input_dict.items():
    char_array.append(key)
    value_counts.append(value)

  value_counts = np.array(value_counts)
  return np.random.choice(char_array, p=list(value_counts/sum(value_counts)))[0]

In [49]:
n=1
onechar = {}
for i in range(len(text)-n-1):
  reference, next_value = text[i:i+n], text[i+n]
  # if it is in dictionary, add one to the next character
  if reference in onechar:
    onechar[reference][next_value] += 1
  # otherwise, give it the initial values
  else:
    onechar[reference] = {x:0 for x in uniq}
    onechar[reference][next_value] += 1

print(f'Number of unique {n} character sequences: {len(onechar)}')

Number of unique 1 character sequences: 65


In [50]:
# Replace the value of n here for the last n characters
n=5
d = {}
for i in range(len(text)-n-1):
  reference, next_value = text[i:i+n], text[i+n]
  # if it is in dictionary, add one to the next character
  if reference in d:
    d[reference][next_value] += 1
  # otherwise, give it the initial values
  else:
    d[reference] = {x:0 for x in uniq}
    d[reference][next_value] += 1

print(f'Number of unique {n} character sequences: {len(d)}')

Number of unique 5 character sequences: 141021


In [51]:
# The distribution of characters given a starting n characters
print(d['ROMEO'])

{'A': 0, 'D': 0, 'o': 0, 'c': 0, '&': 0, '$': 0, 'p': 0, 'u': 0, 'Y': 0, 'r': 0, ';': 0, 'K': 0, 'F': 0, 'U': 0, 'Q': 0, 'N': 0, 'j': 0, 'x': 0, ' ': 0, 'Z': 0, 'B': 0, 'y': 0, 'E': 0, 'l': 0, 'q': 0, 'e': 0, ',': 0, 'm': 0, 'T': 0, 'X': 0, '.': 0, 'f': 0, 'v': 0, 'h': 0, 'W': 0, 'z': 0, '?': 0, '-': 0, 'k': 0, 'P': 0, 'd': 0, 'V': 0, 'i': 0, 'O': 0, '\n': 0, 'g': 0, '!': 0, 'w': 0, 'C': 0, 'n': 0, '3': 0, ':': 163, 's': 0, "'": 0, 'b': 0, 'a': 0, 'S': 0, 'J': 0, 'M': 0, 'R': 0, 't': 0, 'G': 0, 'I': 0, 'H': 0, 'L': 0}


Now we can then sample and update the next 1000 characters

In [52]:
myinput = "ROMEO"
for i in range(1000):
  # if there is something seen before, go by the probability distribution
  if myinput[-n:] in d:
    myinput += sample(d[myinput[-n:]])
  # if not, then just random sample based on just one character
  else:
    myinput += sample(onechar[myinput[-1]])

print(myinput)


ROMEO:
Is Lewis beauty throught you, in the king.

FLORIZEL:
We were he is stand fast;
Read in this knife
Shouting on fortune cannot loss shame that thou art, his oath three despair:
I will not be ruled, form your bring Baptista's daught o'erlook up the world ran
From my cares you wilt thou lions
in thy conquer'd; so poor and more amazed; but on sudden medlar;
My sovereign thy gold;
The fee'd, I'll tell thee mine own great seem'd, is good Grumio!

JULIET:
Mine own but twenty years, his pity, which received off again tune.
Behold me to such less to my mother, whose stol'n them.

BUCKINGHAM:
He wounds!
I was a most censures in the list: but judgment no other there,
As cheer;
But owes
With our voice amazedly; and yourself.
If gracious man, sweet Warwick's daughters, leave of thy title,
The seal o' the midst
John Naps of Juliet ere thee at they have strength! a brother; and men,
And in King Edward's rob our gracious slain:
Wine, is Warwick, her foolish thee
To merit,
When thou shall I have